In [1]:
import pandas as pd
import numpy as np
import csv

In [12]:
stations = pd.read_csv("../Data/station.csv")
status = pd.read_csv("../Data/status.csv")
trip = pd.read_csv("../Data/trip.csv")
weather = pd.read_csv("../Data/weather.csv")

## How does the data look like?

In [13]:
stations.head(5)

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013


Stations table has 7 fields with station information - location (lat, long, city), dock count and installation date are the main columns.

In [14]:
status.head(5)

,station_id,bikes_available,docks_available,time
0,2,2,25,2013/08/29 12:06:01
1,2,2,25,2013/08/29 12:07:01
2,2,2,25,2013/08/29 12:08:01
3,2,2,25,2013/08/29 12:09:01
4,2,2,25,2013/08/29 12:10:01


Status table has 4 columns with info of number of bikes available at a station for a given time. Further, looks like it is updated every minute.

In [15]:
trip.head(5)

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4576,63,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127
1,4607,70,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138
2,4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214
3,4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060
4,4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103


Trip table has 11 fields with trip info - start and stop info, what time and which station, subscriber type, bike id.

In [17]:
weather.head(5)

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
0,8/29/2013,74.0,68.0,61.0,61.0,58.0,56.0,93.0,75.0,57.0,...,10.0,10.0,23.0,11.0,28.0,0,4.0,NaN,286.0,94107
1,8/30/2013,78.0,69.0,60.0,61.0,58.0,56.0,90.0,70.0,50.0,...,10.0,7.0,29.0,13.0,35.0,0,2.0,NaN,291.0,94107
2,8/31/2013,71.0,64.0,57.0,57.0,56.0,54.0,93.0,75.0,57.0,...,10.0,10.0,26.0,15.0,31.0,0,4.0,NaN,284.0,94107
3,9/1/2013,74.0,66.0,58.0,60.0,56.0,53.0,87.0,68.0,49.0,...,10.0,10.0,25.0,13.0,29.0,0,4.0,NaN,284.0,94107
4,9/2/2013,75.0,69.0,62.0,61.0,60.0,58.0,93.0,77.0,61.0,...,10.0,6.0,23.0,12.0,30.0,0,6.0,NaN,277.0,94107


Weather info - min/max temperature, dew point and humidity.

## Possible analyses:

Time series based predictions:
- Predict number of trips per station/day. Further can be extended to show demand vs supply at these stations.
- Predict travel time for station/day. Is the current pricing strategy which is maximum usage of 30 minutes apt?

Clustering:
- Cluster stations. What are the primary traits common to stations in one cluster? This gives us factors that stand out - demand? location? number of docks? installation date? travel time?

Recommendation based:
- Recommend locations where new stations can be installed based on current supply/demand. (Prediction component not necessarily required)


## Using SparkSQL

In [23]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer

sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [24]:
path = "/Users/deena/Documents/Intersession/ProjectWork/"
station = sqlContext.read.load(path + "Data/station.csv", 
                      format='com.databricks.spark.csv', 
                      header='true', 
                      inferSchema='true')

status = sqlContext.read.load(path + "Data/status.csv", 
                      format='com.databricks.spark.csv', 
                      header='true', 
                      inferSchema='true')

trip = sqlContext.read.load(path + "Data/trip.csv", 
                      format='com.databricks.spark.csv', 
                      header='true',
                      inferSchema='true')

weather = sqlContext.read.load(path + "Data/weather.csv", 
                      format='com.databricks.spark.csv', 
                      header='true', 
                      inferSchema='true')

In [25]:
sqlContext.sql("""drop table station""")
sqlContext.sql("""drop table status""")
sqlContext.sql("""drop table trip""")
sqlContext.sql("""drop table weather""")

DataFrame[]

In [26]:
station.write.saveAsTable("station")
status.write.saveAsTable("status")
trip.write.saveAsTable("trip")
weather.write.saveAsTable("weather")

In [27]:
# Station table
sqlContext.sql("""select id, name, lat, long, dock_count, city, 
    DATEDIFF(cast('2017-01-01 00:00:00' as TIMESTAMP), installation_date) as age from station limit 2""").collect()

[Row(id=2, name=u'San Jose Diridon Caltrain Station', lat=37.329732, long=-121.901782, dock_count=27, city=u'San Jose', age=1244),
 Row(id=3, name=u'San Jose Civic Center', lat=37.330698, long=-121.888979, dock_count=15, city=u'San Jose', age=1245)]

In [37]:
# Status table
sqlContext.sql("""select station_id, hour(time) as hour, 
(case when date_format(time, 'EEEE') in ('Saturday', 'Sunday') then 1 else 0 end) as weekday_flag, 
avg(docks_available) as docks_available,
avg(bikes_available) as bikes_available
from status group by 1,2,3 limit 2""").collect()

[Row(station_id=5, hour=12, weekday_flag=1, docks_available=9.879248947879573, bikes_available=9.120751052120427),
 Row(station_id=5, hour=2, weekday_flag=0, docks_available=10.020046260601388, bikes_available=8.979953739398612)]

In [30]:
# Weather table
sqlContext.sql("""select zip_code, month(date) as month, year(date) as year,
(case when date_format(date, 'EEEE') in ('Saturday', 'Sunday') then 1 else 0 end) as weekday_flag,
avg(max_temperature_f) as max_temperature_f, avg(mean_temperature_f) as mean_temperature_f,
avg(min_temperature_f) as min_temperature_f, avg(max_dew_point_f) as max_dew_point_f,
avg(mean_dew_point_f) as mean_dew_point_f, avg(min_dew_point_f) as min_dew_point_f,
avg(max_humidity) as max_humidity, avg(mean_humidity) as mean_humidity,
avg(min_humidity) as min_humidity, avg(max_sea_level_pressure_inches) as max_sea_level_pressure_inches,
avg(mean_sea_level_pressure_inches) as mean_sea_level_pressure_inches,
avg(min_sea_level_pressure_inches) as min_sea_level_pressure_inches,
avg(max_visibility_miles) as max_visibility_miles, avg(mean_visibility_miles) as mean_visibility_miles,
avg(min_visibility_miles) as min_visibility_miles, avg(max_wind_Speed_mph) as max_wind_Speed_mph,
avg(mean_wind_speed_mph) as mean_wind_speed_mph, avg(max_gust_speed_mph) as max_gust_speed_mph,
avg(precipitation_inches) as precipitation_inches, avg(cloud_cover) as cloud_cover,
avg(wind_dir_degrees) as wind_dir_degrees,
avg(case when events = 'Fog' then 1 when (events = 'rain' or events = 'Rain') then 2 when events = 'Fog-Rain' then 3
 when events = 'Rain-Thunderstorm' then 4 else 0 end) as events
from weather group by 1,2,3,4 limit 1""").collect()

[Row(zip_code=94041, month=6, year=2015, weekday_flag=0, max_temperature_f=76.63636363636364, mean_temperature_f=67.0909090909091, min_temperature_f=56.95454545454545, max_dew_point_f=57.81818181818182, mean_dew_point_f=54.77272727272727, min_dew_point_f=51.45454545454545, max_humidity=86.4090909090909, mean_humidity=68.0, min_humidity=49.13636363636363, max_sea_level_pressure_inches=29.97681818181819, mean_sea_level_pressure_inches=29.93409090909091, min_sea_level_pressure_inches=29.886818181818175, max_visibility_miles=10.0, mean_visibility_miles=9.863636363636363, min_visibility_miles=9.136363636363637, max_wind_Speed_mph=17.727272727272727, mean_wind_speed_mph=6.590909090909091, max_gust_speed_mph=23.19047619047619, precipitation_inches=0.005238095238095238, cloud_cover=1.5909090909090908, wind_dir_degrees=230.4090909090909, events=0.09090909090909091)]

## Final combined dataset

In [27]:
final_df = sqlContext.sql("""select W.station_id, W.weekday_flag, W.year, W.month, W.hour,
    X.dock_count, X.name, X.age, X.lat, X.long, X.city,        
    Y.mean_temperature_f, Y.max_dew_point_f, Y.min_dew_point_f,
    Y.mean_humidity, Y.max_sea_level_pressure_inches, Y.mean_sea_level_pressure_inches,
    Y.min_sea_level_pressure_inches, Y.max_visibility_miles, Y.mean_visibility_miles,
    Y.min_visibility_miles, Y.max_wind_Speed_mph, Y.mean_wind_speed_mph, Y.max_gust_speed_mph,
    Y.precipitation_inches, Y.cloud_cover, Y.wind_dir_degrees, Y.events,
    incoming_bikes-outgoing_bikes as bike_traffic
    from
    (
        select cast(zip_code as int) as zipcode, a.start_station_id as station_id, a.hour,
        (case when date_format(start_date, 'EEEE') in ('Saturday', 'Sunday') then 1 else 0 end) as weekday_flag,
        month(start_date) as month, year(start_date) as year,
        avg(outgoing_bikes) as outgoing_bikes, avg(incoming_bikes) as incoming_bikes
        from
        (
            select zip_code, hour(start_date) as hour, start_station_id, cast(start_date as DATE) as start_date,
            avg(duration) as avg_duration, sum(1) as outgoing_bikes
            from trip
            group by 1,2,3,4
        ) as a
        left join
        (
            select hour(end_date) as hour, end_station_id, cast(end_date as DATE) as end_date, sum(1) as incoming_bikes
            from trip
            group by 1,2,3
        ) as b
        on a.start_station_id = b.end_station_id and a.hour = b.hour and a.start_date = b.end_date
        group by 1,2,3,4,5,6
    ) as W
    left join
    (
        select id, name, lat, long, dock_count, city, 
        DATEDIFF(cast('2017-01-01 00:00:00' as TIMESTAMP), installation_date) as age 
        from station
    ) as X
    on W.station_id = X.id
    left join
    (
        select cast(zip_code as int) as zipcode, month(date) as month, year(date) as year,
        (case when date_format(date, 'EEEE') in ('Saturday', 'Sunday') then 1 else 0 end) as weekday_flag,
        avg(max_temperature_f) as max_temperature_f, avg(mean_temperature_f) as mean_temperature_f,
        avg(min_temperature_f) as min_temperature_f, avg(max_dew_point_f) as max_dew_point_f,
        avg(mean_dew_point_f) as mean_dew_point_f, avg(min_dew_point_f) as min_dew_point_f,
        avg(max_humidity) as max_humidity, avg(mean_humidity) as mean_humidity,
        avg(min_humidity) as min_humidity, avg(max_sea_level_pressure_inches) as max_sea_level_pressure_inches,
        avg(mean_sea_level_pressure_inches) as mean_sea_level_pressure_inches,
        avg(min_sea_level_pressure_inches) as min_sea_level_pressure_inches,
        avg(max_visibility_miles) as max_visibility_miles, avg(mean_visibility_miles) as mean_visibility_miles,
        avg(min_visibility_miles) as min_visibility_miles, avg(max_wind_Speed_mph) as max_wind_Speed_mph,
        avg(mean_wind_speed_mph) as mean_wind_speed_mph, avg(max_gust_speed_mph) as max_gust_speed_mph,
        avg(precipitation_inches) as precipitation_inches, avg(cloud_cover) as cloud_cover,
        avg(wind_dir_degrees) as wind_dir_degrees,
        avg(case when events = 'Fog' then 1 when (events = 'rain' or events = 'Rain') then 2 when events = 'Fog-Rain' then 3
         when events = 'Rain-Thunderstorm' then 4 else 0 end) as events
        from weather 
        group by 1,2,3,4
    ) as Y
    on W.zipcode=Y.zipcode and W.weekday_flag=Y.weekday_flag and W.month=Y.month and W.year=Y.year
    left join
    (
        select station_id, hour(time) as hour, month(time) as month, year(time) as year,
        (case when date_format(time, 'EEEE') in ('Saturday', 'Sunday') then 1 else 0 end) as weekday_flag, 
        avg(docks_available) as docks_available,
        avg(bikes_available) as bikes_available
        from status 
        group by 1,2,3,4,5
    ) as Z
    on W.station_id=Z.station_id and W.hour=Z.hour and W.weekday_flag=Z.weekday_flag and W.month=Z.month and W.year=Z.year
""")

In [28]:
final_df.printSchema()

root
 |-- station_id: integer (nullable = true)
 |-- weekday_flag: integer (nullable = false)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city: string (nullable = true)
 |-- mean_temperature_f: double (nullable = true)
 |-- max_dew_point_f: double (nullable = true)
 |-- min_dew_point_f: double (nullable = true)
 |-- mean_humidity: double (nullable = true)
 |-- max_sea_level_pressure_inches: double (nullable = true)
 |-- mean_sea_level_pressure_inches: double (nullable = true)
 |-- min_sea_level_pressure_inches: double (nullable = true)
 |-- max_visibility_miles: double (nullable = true)
 |-- mean_visibility_miles: double (nullable = true)
 |-- min_visibility_miles: double (nullable = true)
 |-- max_wind_Speed_mph: double (nullable 

In [29]:
#final_df.where(col("hour").isNull()).show()
final_df.take(1)

[Row(station_id=72, weekday_flag=0, year=2013, month=11, hour=8, dock_count=23, name=u'Civic Center BART (7th at Market)', age=1227, lat=37.781039, long=-122.411748, city=u'San Francisco', mean_temperature_f=None, max_dew_point_f=None, min_dew_point_f=None, mean_humidity=None, max_sea_level_pressure_inches=None, mean_sea_level_pressure_inches=None, min_sea_level_pressure_inches=None, max_visibility_miles=None, mean_visibility_miles=None, min_visibility_miles=None, max_wind_Speed_mph=None, mean_wind_speed_mph=None, max_gust_speed_mph=None, precipitation_inches=None, cloud_cover=None, wind_dir_degrees=None, events=None, bike_traffic=0.8181818181818181)]

In [45]:
final_df.write.saveAsTable("final_df")

## Model Building

In [31]:
# Fill NAs
final_df = final_df.fillna(0)
final_df.show()

+----------+------------+----+-----+----+----------+--------------------+----+---------+-----------+-------------+------------------+---------------+---------------+-------------+-----------------------------+------------------------------+-----------------------------+--------------------+---------------------+--------------------+------------------+-------------------+------------------+--------------------+-----------+----------------+------+------------------+
|station_id|weekday_flag|year|month|hour|dock_count|                name| age|      lat|       long|         city|mean_temperature_f|max_dew_point_f|min_dew_point_f|mean_humidity|max_sea_level_pressure_inches|mean_sea_level_pressure_inches|min_sea_level_pressure_inches|max_visibility_miles|mean_visibility_miles|min_visibility_miles|max_wind_Speed_mph|mean_wind_speed_mph|max_gust_speed_mph|precipitation_inches|cloud_cover|wind_dir_degrees|events|      bike_traffic|
+----------+------------+----+-----+----+----------+----------

In [32]:
#converting strings to numeric values
from pyspark.ml.feature import StringIndexer

def indexStringColumns(df, cols):
    #variable newdf will be updated several times
    newdf = df
    
    for c in cols:
        #For each given colum, fits StringIndexerModel.
        si = StringIndexer(inputCol=c, outputCol=c+"-num")
        sm = si.fit(newdf)
        #Creates a DataFame by putting the transformed values in the new colum with suffix "-num" 
        #and then drops the original columns.
        #and drop the "-num" suffix. 
        newdf = sm.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-num", c)
    return newdf

dfnumeric = indexStringColumns(final_df, ["name", "city"])

In [33]:
# Merging the data with Vector Assembler.
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(outputCol="features", inputCols=dfnumeric.schema.names)
#lpoints - labeled data.
lpoints = va.transform(dfnumeric).select("features", "bike_traffic").withColumnRenamed("bike_traffic", "label")

In [43]:
lpoints.show()

+--------------------+------------------+
|            features|             label|
+--------------------+------------------+
|(29,[0,2,3,4,5,6,...|0.8181818181818181|
|(29,[0,2,3,4,5,6,...|               0.0|
|(29,[0,2,3,4,5,6,...|0.6666666666666667|
|(29,[0,2,3,4,5,6,...|               2.0|
|(29,[0,2,3,4,5,6,...|              0.25|
|(29,[0,2,3,4,5,6,...|               0.0|
|(29,[0,2,3,4,5,6,...|               3.0|
|(29,[0,2,3,4,5,6,...|               0.0|
|(29,[0,2,3,4,5,6,...| 5.636363636363637|
|(29,[0,2,3,4,5,6,...|               0.0|
|(29,[0,2,3,4,5,6,...|               7.0|
|(29,[0,2,3,4,5,6,...|               0.0|
|(29,[0,2,3,4,5,6,...|1.2999999999999998|
|(29,[0,2,3,4,5,6,...|               0.0|
|(29,[0,2,3,4,5,6,...|              1.55|
|(29,[0,2,3,4,5,6,...|              0.75|
|(29,[0,2,3,4,5,6,...|               1.0|
|(29,[0,2,3,4,5,6,...|               0.0|
|(29,[0,1,2,3,4,5,...|               0.0|
|(29,[0,2,3,4,5,6,...| 8.545454545454545|
+--------------------+------------

In [40]:
#Divide the dataset into training and testing sets.
splits = lpoints.randomSplit([0.8, 0.2])

#cache() : the algorithm is interative and training and data sets are going to be reused many times.
train_df = splits[0].cache()
valid_df = splits[1].cache()

In [44]:
#Train the model.
from pyspark.ml.linalg import Vectors
lr = LinearRegression(maxIter=5, regParam=0.0, solver="normal", weightCol="weight")
model = lr.fit(train_df)

ImportError: cannot import name LinearRegression

In [42]:
lrmodel = lr.fit(train_df)

Py4JJavaError: An error occurred while calling o402.fit.
: org.apache.spark.SparkException: Classification labels should be in [0 to 54]. Found 40112 invalid labels.
	at org.apache.spark.ml.classification.LogisticRegression.train(LogisticRegression.scala:563)
	at org.apache.spark.ml.classification.LogisticRegression.train(LogisticRegression.scala:487)
	at org.apache.spark.ml.classification.LogisticRegression.train(LogisticRegression.scala:278)
	at org.apache.spark.ml.Predictor.fit(Predictor.scala:118)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:280)
	at py4j.commands.Abstract